# stocke les episodes

In [ ]:
import pymongo
import feedparser
from datetime import datetime


def stocke_episodes(url_podcast):
    # Connexion à la base de données MongoDB
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client["masque_et_la_plume"]
    collection = db["episodes"]

    # Récupération du flux RSS
    feed = feedparser.parse(url_podcast)

    # Parcours des items du flux RSS
    for entry in feed.entries:
        nom = entry.title
        publié = entry.published
        duree = entry.itunes_duration if "itunes_duration" in entry else None
        url = entry.enclosures[0].href if entry.enclosures else None
        description = entry.description
        auteur = entry.author if "author" in entry else None

        # Conversion de la date de publication
        publié = datetime.strptime(publié, "%a, %d %b %Y %H:%M:%S %z")

        # Vérification de la durée de l'épisode
        if duree:
            duree_parts = duree.split(":")
            if len(duree_parts) == 3:
                duree_seconds = (
                    int(duree_parts[0]) * 3600
                    + int(duree_parts[1]) * 60
                    + int(duree_parts[2])
                )
            elif len(duree_parts) == 2:
                duree_seconds = int(duree_parts[0]) * 60 + int(duree_parts[1])
            else:
                duree_seconds = int(duree_parts[0])
        else:
            duree_seconds = 0

        if duree_seconds > 900:
            # Vérification si l'épisode est déjà présent dans la base
            if collection.find_one({"nom": nom, "publié": publié}) is None:
                # Création du document pour l'épisode
                episode = {
                    "nom": nom,
                    "publié": publié,
                    "duree": duree_seconds,
                    "url": url,
                    "description": description,
                    "auteur": auteur,
                }

                # Insertion de l'épisode dans la collection
                collection.insert_one(episode)
                print(f"Épisode inséré: {nom}")
            else:
                print(f"Épisode déjà présent: {nom}")
        else:
            print(f"Épisode ignoré (durée insuffisante): {nom}")


# Exemple d'utilisation
url_podcast = "https://radiofrance-podcast.net/podcast09/rss_14007.xml"
stocke_episodes(url_podcast)

Épisode déjà présent: "Sarah Bernhardt, la divine", "Les Femmes aux balcon", "Vingt Dieux", "Oh Canada"... Que voir au cinéma ?
Épisode ignoré (durée insuffisante): "Oh, Canada" de Paul Schrader
Épisode ignoré (durée insuffisante): "Vingt Dieux" de Louise Courvoisier : le Masque mi-charmé, mi-irrité
Épisode ignoré (durée insuffisante): "Saint-Ex" de Pablo Agüero : un film en carton pâte et à la poésie plate pour les critiques du Masque
Épisode ignoré (durée insuffisante): "Les Femmes au balcon" : Noémie Merlant signe un film féministe audacieux mais trop superficiel
Épisode ignoré (durée insuffisante): "Sarah Bernhardt, la divine" : Sandrine Kiberlain brille et sauve un film "trop théâtral et exagéré"
Épisode déjà présent: Les nouvelles pages de Marc Dugain, Emmanuelle Lambert, Emil Ferris, Fabrice Caro et Mathieu Palain
Épisode ignoré (durée insuffisante): "L’Avion, Poutine, l’Amérique... et moi", de Marc Dugain : le Masque majoritairement assez sévère
Épisode ignoré (durée insuffisan

# sauve la base

In [ ]:
!mongodump --db masque_et_la_plume --out /home/guillaume/git/lmelp/db

2024-12-30T19:18:28.353+0100	writing masque_et_la_plume.episodes to /home/guillaume/git/lmelp/db/masque_et_la_plume/episodes.bson
2024-12-30T19:18:28.354+0100	done dumping masque_et_la_plume.episodes (9 documents)
